## The aim of this notebook is to calculate a cost estimation of different methods to calculate the energy of a system with different Phase Estimation protocols

In [2]:
import numpy as np
from itertools import combinations
import scipy
from scipy.optimize import minimize
from scipy.special import binom, gamma
from scipy.integrate import quad, dblquad
from scipy import integrate
import sympy

## IMPORTANT: to these cost, we have to add the QFT cost, which is minor, and has an associated error.

Question: what is the base of the logs that appear?

# qDrift and Trotterization

The first algorithm we would like to estimate its complexity is the q-Drift protocol from appendix A. The cost is 
$$3n(10+12\log \epsilon^{-1})\log N  = 3\frac{27\pi^2}{2}\frac{\lambda^2}{\delta_E^2 P_f}(10+12\log \epsilon^{-1})\log N$$
where $\lambda = \sum a_\gamma $ for the Hamiltonian $H = a_\gamma H_\gamma$;
$\delta_E$ is the error in Phase Estimation (an arbitrary parameter chosen by the user);
and $P_f = \frac{3}{2}p_f$ the probability of failure (also chosen by the user). The $\epsilon$ parameter is given by the smallest of
$$\epsilon_j = \epsilon_{tot}\frac{2^j}{2(2^m-1)};$$ 
We also need that
$$ n = 4\frac{\pi^2(2^m-1)^2}{\epsilon_{tot}}$$ with

$$m = q +\log_2 \left(\frac{1}{2p_f} + \frac{1}{2}\right),$$
$\delta_E = 2\lambda\delta$,  $q = \log_2 \delta -1$; and $P_f = p_f +2\epsilon_{tot}$

We want to minimize the total cost
$$3n(10+12\log \epsilon^{-1})\log N  = 3\frac{27\pi^2}{2}\frac{\lambda^2}{\delta_E^2 P_f}(10+12\log \epsilon^{-1})\log N$$
where $\epsilon$ is the error of individual rotations $$\epsilon = \frac{\epsilon(j)}{n(j)} = \frac{\epsilon_{tot}^2}{4\pi^2(2^m-1)^2}$$

In [27]:
def calc_qdrift_resources(lambd, N, deltaE = 1e-4, P_failure = .1):
    n = ((27*np.pi**2/2)*(lambd/deltaE)**2) / P_failure**3
    
    delta = deltaE/(2*lambd)
    q = np.log2(1/delta)-1
    
    pf = 2/3*P_failure
    eps_tot = P_failure/6
    #sanity check
    assert (pf +2*eps_tot)/P_failure == 1
    
    m = q + np.log2(1/(2*pf)+1/2)
    
    # Another sanity check. This should coincide
    eps_tot_2 = 4*(np.pi*2**m-1)**2/n
    print(eps_tot,eps_tot_2)
    
    # error in individual rotations
    eps = (eps_tot/(2*np.pi*(2**m-1)))**2

    rost_cost_factor = 3*(10+12*np.log(1/eps))*np.log(N)
    print('eps',eps)
    
    return rost_cost_factor*n

calc_qdrift_resources(lambd = 768, N = (467403)**(1/4))

0.016666666666666666 0.01896296276647537
eps 1.8639547394483464e-21


4.484316495050195e+22

For the randomised Hamiltonian approach, the equations are similar. However, now $p_f = 3/4P_f$ and 
$$n = 8\Gamma^2\left(\frac{ \pi^3 \Lambda^3}{8\delta_E^3}\right)^{1/2}\left(\frac{1+p_f}{p_f}\right)^{3/2}\frac{1}{\epsilon_{tot}^{1/2}} = 4.35\sqrt{8}\pi^{3/2}\Gamma^2 \frac{\Lambda^{3/2}}{\delta_E^{3/2}P_f^2}$$ 

In [9]:
def calc_rand_ham_resources(Lambd, lambd, Gamma, N, deltaE = 1e-4, P_failure = .1):
    n = 4.35*np.sqrt(8)*(np.pi*Lambd/deltaE)**(3/2) *(Gamma/ P_failure)**2
    print('n',n)

    # error in individual rotations
    Lambda_A = Lambd/(2*lambd)
    delta = deltaE/(2*lambd)
    q = np.log2(1/delta)-1
    
    pf = 3/4*P_failure
    eps_tot = P_failure/8
    #sanity check
    assert (pf +2*eps_tot)/P_failure == 1
    
    m = q + np.log2(1/(2*pf)+1/2)
    
    n1 = 8*Gamma**2 * ( 2**(m+1)*np.pi**3*Lambda_A**3/eps_tot  )**(1/2) *2*(2**m-1)
    print('n1',n1)
    
    # Another sanity check. This should coincide
    eps_tot_2 = ( 8*Gamma**2* (np.pi*Lambd/(2*deltaE))**(3/2)* ((1+pf)/pf)**(3/2) /n1    )**2
    eps_tot_3 = 1/ ( 4.35* (1/P_failure)**2  *  (pf/(1+pf))**(3/2)  )**2
    print(eps_tot,eps_tot_2,eps_tot_3)
    
    n2 = 8*Gamma**2 * ( 2**(m+1)*np.pi**3*Lambda_A**3/eps_tot_2  )**(1/2) *2*(2**m-1)
    print('n2',n2)
    
    n3 = 8*Gamma**2 * ( 2**(m+1)*np.pi**3*Lambda_A**3/eps_tot_3  )**(1/2) *2*(2**m-1)
    print('n3',n3)

    
    # Esto probablemente esté mal:
    eps = 1/4*(eps_tot/(np.pi*2**m*Lambda_A))**(3/2)

    rost_cost_factor = 3*(10+12*np.log(1/eps))*np.log(N)
    print('eps',eps)
    
    return rost_cost_factor*n

calc_rand_ham_resources(Lambd = 4.07, lambd = 768, Gamma =467403,  N = (467403)**(1/4))

n 1.2289484451445014e+22
n1 1.3712296468152677e+22
0.0125 0.012500000454215137 0.015561916785328065
n2 1.3712296219019378e+22
n3 1.2289484228162224e+22
eps 1.1265689857382676e-12


4.092893794257138e+25

# Taylorization (babbush2016exponential)

Let us know calcula the cost of performing Phase Estimation. 
1.  We have already mentioned that in this case, controlling the direction of the time evolution adds negligible cost. We will also take the unitary $U$ in Phase estimation to be $U_r$. The number of segments we will have to Hamiltonian simulate in the phase estimation protocol is $r \approx \frac{4.7}{\epsilon_{\text{PEA}}}$.
2. Using oblivious amplitude amplification operator $G$ requires to use $\mathcal{W}$ three times.
3. Each operator $G$ requires to use Prepare$(\beta)$ twice and Select$(V)$ once.
4. The cost of Select$(V)$ is bounded in $8N\lceil \log_2\Gamma + 1\rceil\frac{K(K+1)(2K+1)}{3}+ 16N K(K+1)$.
5. The cost of Prepare$(\beta)$ is $(20+24\log\epsilon^{-1}_{SS})K$ T gates for the preparation of $\ket{k}$; and $(10+12\log\epsilon^{-1}_{SS})2^{\lceil \log \Gamma \rceil + 1}K$ T gates for the implementation of the $K$ Prepare$(W)$ circuits. Here notice that $2K$ and $2^{\lceil \log \Gamma \rceil + 1}K$ rotations up to error $\epsilon_{SS}$ will be implemented.
Remember that 
$$ K =  O\left( \frac{\log(r/\epsilon_{HS})}{\log \log(r/\epsilon_{HS})} \right)$$
Notice that the $\Lambda$ parameters comes in the algorithm only implicitly, since we take the evolution time of a single segment to be $t_1 = \ln 2/\Lambda$ such that the first segment in Phase estimation has $r = \frac{\Lambda t_1}{\ln 2} = 1$ as it should be. In general, we will need to implement $r \approx \frac{4.7}{\epsilon_{PEA}}$. However, since $\epsilon_{PEA}$ makes reference to $H$ and we are instead simulating $H \ln 2/ \Lambda$, we will have to calculate the eigenvalue to precision $\epsilon \ln 2/ \Lambda$; so it is equivalently to fixing an initial time $t_1$ and running multiple segments in each of the $U$ operators in Phase Estimation.

In [31]:
def Taylor_naive(Lambd, Gamma, N, epsilon_PEA = .4*eps_tot, epsilon_HS = .2*eps_tot, epsilon_S = .4*eps_tot):
    
    r = 4.7*Lambd / (epsilon_PEA*np.log(2)) # The simulated time
    
    K_list = []
    
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * 2**m_j
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))
        K_list.append(K)
        
    result = 0
    
    epsilon_SS = epsilon_S /(np.sum([3*2*(K*2**(np.ceil(np.log2(Gamma)+1)) + 2*K) for K in K_list]))
        
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * t_j
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))    
        Select_V = 8*N*np.ceil(np.log2(Gamma) +1)*K*(K+1)*(2*K+1)/3 + 16*N*K*(K+1)

        Prepare_beta_1 = (20+24*np.log2(1/epsilon_SS))*K
        Prepare_beta_2 = (10+12*np.log2(1/epsilon_SS))*K*2**(np.ceil(np.log2(Gamma)+1))
        Prepare_beta = Prepare_beta_1 + Prepare_beta_2
        
        result += 3*(2*Prepare_beta + Select_V)*t_j
        
    return result

Taylor_naive(Lambd = 4.07, Gamma =467403, N = (467403)**(1/4))

95463557056106.14

In [6]:
eps_tot = .0125
def Taylor_on_the_fly(Gamma, N, phi_max, dphi_max, zeta_max_i, epsilon_PEA = .4*eps_tot, epsilon_HS = .1*eps_tot, epsilon_S = .4*eps_tot, epsilon_H = .1*eps_tot, order = 10):
    '''
    Error terms 
    eps_PEA: Phase estimation, 
    eps_HS: the truncation of K, 
    eps_S: gate synthesis, 
    eps_H: discretization of integrals,
    eps_taylor: truncation of taylor series to order o
    '''

    t = 4.7/epsilon_PEA
    x_max = np.log(N * t/ epsilon_H)
    
    lambd = Gamma*phi_max**4 * x_max**5
    r = lambd* t / np.log(2)
    
    K_list = []
    
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * 2**m_j
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))
        K_list.append(K)

    epsilon_SS = epsilon_S /np.sum([3*2*(2*K) for K in K_list])
    
    # We distribute the error between all C-U in phase estimation uniformly
    eps_tay_m_j = eps_tay/((6+2)*np.max(K_list)*r*3*2)

    x = sympy.Symbol('x')
    order = max(order_find(function = sqrt(x), x0 = 1, e = eps_tay_m_j, xeval = x_max), 
                order_find(function = exp(max_zeta_i*(x)**2), x0 = 0, e = eps_tay_m_j, xeval = x_max))
    
    result = 0
    
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * 2**m_j
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))
    
        mu = ( 3*K*2*r/epsilon_H *2*(4*dphi_max + phi_max/x_max)*phi_max**3 * x_max**6 )**6
        n = np.log(mu)/3

        Select_V = 8*N*np.ceil(np.log2(Gamma) +1)*K*(K+1)*(2*K+1)/3 + 16*N*K*(K+1)

        Prepare_beta_1 = (20+24*np.log2(1/epsilon_SS))*K

        Prepare_beta_2 = ( 6*35*n**2*(order-1)*4*N + (252+70*(order-1))*n**2 )*K

        Prepare_beta = Prepare_beta_1 + Prepare_beta_2
        
        result += 3*(2*Prepare_beta + Select_V)*t_j
        
    return result

Taylor_on_the_fly(Gamma = 467403, N = (467403)**(1/4), phi_max = .1, dphi_max = .1)

4.81664159586087e+21

In [5]:
# Taylor approximation at x0 of the function 'function'
def taylor(function,x0,n):
    i = 0
    p = 0
    while i <= n:
        p = p + (function.diff(x,i).subs(x,x0))/(factorial(i))*(x-x0)**i
        i += 1
    return p

#print(taylor(sympy.sqrt(x), 1, 5))#.subs(x,1).evalf())

def order_find(function, x0, e, xeval):
    
    x = sympy.Symbol('x')
    
    def factorial(n):
        if n <= 0:
            return 1
        else:
            return n*factorial(n-1)
    
    def taylor_err(function,x0,n, z = None):
        if z == None:
            z = x0
        #print('coefficient order',n, function.diff(x,n)/(factorial(n)))#.subs(x,z))
        a = (function.diff(x,n).subs(x,z))/(factorial(n))*(x-x0)**n
        #print('coefficient order',n, (function.diff(x,n).subs(x,z)/(factorial(n))*(x-x0)**n))
        #print('a',a)
        return a

    order = 0
    te = 1
    zeta = np.linspace(x0,xeval,20)
    
    while te > e:# or order < 10:
        order += 1
        #for z in zeta:
            #print(taylor_err(f, x0, order, z).subs(x,xeval).evalf())
        te = np.max([np.abs(taylor_err(function, x0, order, z).subs(x,xeval).evalf()) for z in zeta])
        #print('order',order, te,'\n')
        
    return order

x = sympy.Symbol('x')
order_find(sympy.sqrt(x), x0 = 1, e = 1e-3, xeval = 2)

order 1 0.500000000000000 

order 2 0.125000000000000 

order 3 0.0625000000000000 

order 4 0.0390625000000000 

order 5 0.0273437500000000 

order 6 0.0205078125000000 

order 7 0.0161132812500000 

order 8 0.0130920410156250 

order 9 0.0109100341796875 

order 10 0.00927352905273438 

order 11 0.00800895690917969 

order 12 0.00700783729553223 

order 13 0.00619924068450928 

order 14 0.00553503632545471 

order 15 0.00498153269290924 

order 16 0.00451451400294900 

order 17 0.00411617453210056 

order 18 0.00377315998775885 

order 19 0.00347527893609367 

order 20 0.00321463301588665 

order 21 0.00298501637189474 

order 22 0.00278149252835647 

order 23 0.00260009084172452 

order 24 0.00243758516411674 

order 25 0.00229133005426974 

order 26 0.00215913793575417 

order 27 0.00203918582821228 

order 28 0.00192994373027233 

order 29 0.00183011905456859 

order 30 0.00173861310184016 

order 31 0.00165448666142854 

order 32 0.00157693259917408 

order 33 0.00150525384466617

44

In [91]:
33/2048

0.01611328125

In [30]:
eps_tot = .0125
def error_optimizer(eps_array):
    epsilon_PEA = eps_array[0]
    
    epsilon_S = eps_array[1]
    
    epsilon_HS = eps_tot - eps_array[0] - eps_array[1]
    
    return Taylor_naive(Lambd = 4.07, Gamma =467403, N = (467403)**(1/4),
                 epsilon_PEA = epsilon_PEA, epsilon_HS= epsilon_HS, epsilon_S = epsilon_S)

eps_array = [.005, .005]
#A = np.array([1,1])

constraint = scipy.optimize.LinearConstraint(A = np.array([[1,1],[1,0],[0,1]]), lb = [0,0,0], ub = [eps_tot,eps_tot,eps_tot], keep_feasible=True)
minimize(error_optimizer, x0 = eps_array, method='SLSQP', tol=1, constraints = (constraint))
    

     fun: 95463557056106.14
     jac: array([-1.96938319e+16, -6.01179179e+14])
 message: 'Inequality constraints incompatible'
    nfev: 3
     nit: 1
    njev: 1
  status: 4
 success: False
       x: array([0.005, 0.005])

# Configuration interaction (babbush2017exponentially)

\begin{equation}
    \begin{split}
        &\mu M \zeta = \mu \max_{\gamma,\rho}||\aleph_{\rho,\gamma}||_{\max} = \\
        &=\max \left[ \frac{672\pi^2}{\alpha^3}\varphi^4_{\max}x^5_{\max}\log^6 \left(\frac{K_2 \varphi^4_{\max}x^5_{\max}}{\delta}\right)\right.\\
        &, \left.\frac{256\pi^2}{\alpha^3}Z_q\varphi^2_{\max}x^2_{\max}\log^3 \left(\frac{K_1 Z_q\varphi^2_{\max}x^2_{\max}}{\delta}\right),\right.\\
        &\left. \frac{32\gamma_1^2}{\alpha^3}\varphi^2_{\max}x_{\max}\log^3 \left(\frac{K_0\varphi^2_{\max}x_{\max}}{\delta}\right)\right]
    \end{split}
    \end{equation}

In [3]:
eps_tot = 0.125
def configuration_interaction(N, eta, alpha, gamma1, K0, K1, K2, epsilon_PEA = .4*eps_tot, epsilon_HS = .1*eps_tot, epsilon_S = .4*eps_tot, epsilon_H = .1*eps_tot):
    t = 4.7/epsilon_PEA
    x_max = np.log(N * t/ epsilon_HS)
    
    Gamma = binom(eta, 2)*binom(N-eta, 2) + binom(eta,1)*binom(N-eta,1) + 1 # = d
    Zq = eta
    
    '''
    Warning, we have a circular definition here of delta, mu_M_zeta and r.
    In practice we have to find the smallest value of mu_M_zeta compatible with delta:
    mu_M_zeta \leq f( epsilon_H / 3K*2 Gamma t mu_M_zeta), with f the np.max defining mu_M_zeta below
    Due to this complication we distribute the error uniformly accross all C-U which is not optimal
    '''
    
    delta = epsilon_H/(3*r*K) # delta is the error in calculating a single integral. There are 3K r of them in the simulation
     
    # This is an upper bound, not an equality!!!
    mu_M_zeta = np.max([ 
        672*np.pi**2/(alpha**3)*phi_max**4*x_max**5*(np.log(K2*phi_max**4*x_max**5/delta))**6,
        256*np.pi**2/(alpha**3)*Zq*phi_max**2*x_max**2*(np.log(K1*Zq*phi_max**2*x_max**2/delta))**3,
        32*gamma1**2**2/(alpha**3)*phi_max**2*x_max*(np.log(K0*phi_max**2*x_max/delta))**3
    ])
    
    r = 2*Gamma*t*mu_M_zeta
    K = np.log2(r/epsilon_HS)/np.log2(np.log2(r/epsilon_HS))
    epsilon_SS = epsilon_S / (2*K*2*3*r)
    Prepare_beta = (20+24*np.log2(1/epsilon_SS))*K
        
    mu = ( r/epsilon_H *2*(4*dphi_max + phi_max/x_max)*phi_max**3 * x_max**6 )**6
    n = np.log(mu)/3
    Sample_w = ( 6*35*n**2*(order-1)*4*N + (189+35*(order-1))*n**2 )*K

    Q_val = 2*Sample_w
    Q_col = 6*(32*eta*np.log2(N) + 24*eta**2 + 16*eta*(eta+1)*np.log2(N))
    
    Select_H = Q_val + 2*Q_col
    Select_V = K*Select_H
    
    return r*3*(2*Prepare_beta + Select_V)

# Low depth quantum simulation of materials (babbush2018low) Trotter

In [16]:
def low_depth_trotter(N, Omega, eps_PEA, eps_HS, eps_S):
    def f(x, y):
        return 1/(x**2 + y**2)
    def I(N0):
        return integrate.nquad(f, [[1, N0],[1, N0]])[0]
    
    t = 4.7/eps_PEA
    sum_1_nu = 4*np.pi(np.sqrt(3)*N**(1/3)/2 - 1) + 3 - 3/N**(1/3) + 3*I(N**(1/3))
    max_V = eta**2/(2*np.pi*Omega**(1/3))*sum_1_nu
    max_U = eta**2/(np.pi*Omega**(1/3))*sum_1_nu
    nu_max = 3*(N**(1/3))**2
    max_T = 2*np.pi**2*eta/(Omega**(2/3))* nu_max
    
    r = np.sqrt(2*t**3/eps_HS *(max_T**2*(max_U + max_V) + max_T*(max_U + max_V)**2))
    
    eps_SS = eps_S/(2N +N*(N-1) + N*np.log(N/2) + 8*N)
    
    exp_UV_cost = (4*N**2 + 4*N)*np.log(1/eps_SS)
    FFFT_cost = (2 + 4*np.log(1/eps_SS))*n*np.log(N) + 4*N*np.log(1/eps_SS)
    exp_T_cost = 32*N*np.log(1/eps_SS)
    
    return r*(exp_UV_cost + FFFT_cost + exp_T_cost)

(16.5877713181052, 2.1199305958404497e-06)

from sympy import * 
x,y= symbols('x y') 
a = symbols('a', positive=True)
expr=1/(x**2 + y**2)
integral = integrate(expr,(x,1,a),(y,1,a))
integral
integral.evalf(subs={a:10^6})

# Low depth quantum simulation of materials (babbush2018low) Taylor

In [ ]:
def low_depth_taylor(N, lambd, Lambd, eps_PEA, eps_HS, eps_S, Ham_norm):
    '''To be used in plane wave basis'''
    t = 4.7/eps_PEA
    r = t*Lambda/np.log(2)
    
    K_list = []
    
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * 2**m_j
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))
        K_list.append(K)
        
    epsilon_SS = epsilon_S /np.sum([3*2*(2*K) for K in K_list]) # The extra two is because Uniform requires 2 Rz gates
    
    mu = np.ceil(np.log(2*np.sqrt(2)*Lambdd/eps_PEA) + np.log(1 + eps_PEA/(8*lambd)) + np.log(1 - (Ham_norm/lambd)**2))
    
    result = 0
    
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * 2**m_j
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))
    
        prepare_beta = K*(6*N+40*np.log(N)+16*np.log(1/epsilon_SS) + 10*mu)
        select_V = K*(12*N+8*np.log(N))
        
        result += 3*(2*prepare_beta + select_V)*t_j
    
    return result

# Low depth quantum simulation of materials (babbush2018low) On-the fly

In [ ]:
def low_depth_on_the_fly(N, lambd, Omega, eps_PEA, eps_HS, eps_S, Ham_norm, J):
    '''To be used in plane wave basis
    J: Number of atoms
    '''
    Lambd = (2*eta+1)*N**3 / (2*Omega**(1/3)*np.pi)
    t = 4.7/eps_PEA
    r = t*Lambd/np.log(2)
    
    mu = np.ceil(np.log(2*np.sqrt(2)*Lambdd/eps_PEA) + np.log(1 + eps_PEA/(8*lambd)) + np.log(1 - (Ham_norm/lambd)**2))
    #K_list = []
    
    #epsilon_SS = epsilon_S /np.sum([3*2*(2*K) for K in K_list])
        
    x = sympy.Symbol('x')
    order = order_find(function = cos(x), x0 = 1, e = e, xeval = x_max)
    sample_w = 70*np.log(N)**2 + 29* np.log(N) + (21+14)*order/2*np.log(N)**2 + 2*order*np.log(N) + J*(35*order/2 + 63 + 2*order/np.log(N))*np.log(N)**2
    kickback = 32*np.log(mu)    

    result = 0
    
    for m_j in range(0, int(np.ceil(np.log(r)))):
        
        t_j = 2**m_j
        epsilon_HS_mj = epsilon_HS / r * 2**m_j
    
        #K_list.append(K)
    
        K = np.ceil(np.log2(t_j/epsilon_HS_mj) / np.log2( np.log2 (t_j/epsilon_HS_mj)))
    
        prepare_W = 2*sample_w + kickback
        prepare_beta = K*prepare_W
        select_H = (12*N + 8*np.log(N))
        select_V = K*select_H
        
        result += 3*(2*prepare_beta + select_V)*t_j
    
    return result

## Linear T complexity (babbush2018encoding)

In [ ]:
def linear_T(N, lambd, eps_PEA, eps_SS):
    '''To be used in plane wave basis'''
    t = 4.7/eps_PEA
    r = lambd*t
    
    mu = np.ceil(np.log(2*np.sqrt(2)*lambd/eps_PEA) + np.log(1 + eps_PEA/(8*lambd)) + np.log(1 - (Ham_norm/lambd)**2))
    
    eps_SS = eps_S / (r*2*P)
    
    S = 12*N+8*np.log(N)
    P = 6*N + 40*np.log(N)+ 24*np.log(1/eps_SS) + 10*mu
    
    return r*(2*P + S)

## Sparsity and low rank factorization (berry2019qubitization)

In [2]:
def sparsity_low_rank(N, lambd, eps_PEA, eps_SS, L):
    t = 4.7/eps_PEA
    r = lambd*t
    
    mu = np.ceil(np.log(2*np.sqrt(2)*lambd/eps_PEA) + np.log(1 + eps_PEA/(8*lambd)) + np.log(1 - (Ham_norm/lambd)**2))
    d = L(N**2/8 + N/4)
    M = np.log(N**2) + mu
    
    def closest_power(x):
        possible_results = np.floor(np.log2(x)), np.ceil(np.log2(x))
        return min(possible_results, key= lambda z: abs(x-2**z))
    
    kc = 2**closest_power(np.sqrt(d/M))
    ku = 2**closest_power(np.sqrt(d))
    
    QROAM = 4*(np.ceil(d/kc)+4*M*(kc-1)+2*np.ceil(d/ku) + 4*k_u)
    
    Select = (4*N + 4*np.log(N))*4 # The *4 because Toffoli -> T-gates
    
    # 7 times per prepare, we have to use Uniform
    eps_SS = eps_S/ (7*2*r)
    Uniform = 8*np.log(L) + 56*np.log(1/eps_SS) + 52*np.log(N/2) ### Warning, this is in T gates already!!!!
    
    Other_subprepare = mu + np.log(L) + 6*np.log(N/2)
    
    continuous_register = 2*(np.log(N/2))**2 + 3*np.log(N/2)
    
    Prepare = 4*(QROAM + Other_subprepare + continuous_register) + Uniform # The 4 is Toffoli -> T-gates
    
    return r*(2*Prepare + Select)

4.442882938158366

## Interaction picture (low2019hamiltonian)

In [ ]:
def interaction_picture(N, Gamma, lambd_T, lambd_U_V, eps_S, eps_HS, eps_PEA):
    '''
    The number of rotations is very large here:
    Each of the r segments can be simulated as e^{-i(U+V)t} T(e^{-i \int H_I (s) ds})
    - The Time Ordered Dyson series segment is represented by TDS
    - TDS is made of oblivious Amplitude Amplification of TDS_beta: 2x Ref  + 3x TDS_beta
        < TDS_beta is made of COEF DYS_K COEF'
        < DYS_K is made of
            · 4K U operators
            · K Compare and K Swap
            · (3K + 1) ADD operators
            · K HAM-T operators, made of,
                > x2 e^{-i(U+V)t}
                > x2 FFFT
                > x2 Prepare
                > Select
    Also, the e^{-i(U+V)t} is
        > x2 FFFT
        > N log 1/eps_SS Phase operators
        > N Multiplications
    '''

    t = 4.7/eps_PEA
    r = lambd_T*t # lambd_T is necessary to take tau = 1
    
    # Notice that K is a bit different than in other articles because each segment is now its own Taylor series, which has the consequence of larger error
    K = np.ceil( -1  + 2* np.log(2*r/epsilon_HS)/np.log(np.log(2*r/epsilon_HS))  ) # We 
    delta = eps_HS / t # Alternatively we can substitute t by r changing delta in the following line to 1/2. t represents L in the main text (see before eq 21 in the original article)
    tau = 1/np.ceil(2*lambd_T) # tau = t/ np.ceil(2 * lambd_T * t)
    M = np.max(16*tau/delta * (2*lambd_U_V + lambd_T), K**2)
    
    rot_FFFT = 2*N/2*np.log2(N)
    rot_U = 4*K
    rot_COEF = 2**(np.ceil(np.log2(K) + 1))
    rot_prep = 16*N
    
    epsilon_SS = 1e-2
    consistent = False
    while not consistent:
        rot_exp_U_V = rot_FFFT + N*np.log2(1/epsilon_SS) + N
        num_rotations = ((((2*rot_prep + 2* rot_FFFT + 2*np.log(M)*rot_exp_U_V)*K * rot_U) + 2*rot_COEF)*3 + rot_exp_U_V)*r
        proposed_eps_SS = eps_S / num_rotations
        if proposed_eps_SS < epsilon_SS:
            consistent = True
        else:
            epsilon_SS /= 10
            
    # Cost
    
    exp_U_V= 46*N*(np.log(1/eps_SS))**2+8*N + 8*N*np.log2(1/eps_SS)*np.log2(N) + 4*N*np.log(N)
    COEF = rot_COEF * (10 + 12*np.log2(K))
    U = 8*(np.log2(M) + np.log2(1/eps_SS))
    ADD = 4*np.log2(K)
    Comp = 8*np.log2(M)
    FFFT = (2 + 4*np.log(1/eps_SS))*N*np.log2(N) - 4*np.log2(1/eps_SS)*N
    Prep = 2**9*(1 + np.log2(N))+2**6*3*N*np.log2(1/eps_SS)
    Select = 8*N
    REF = 16*(np.log2(Gamma) + 2*np.log(K+1)+ 2*np.log(M))
    
    cost = ((((2*Prep + Select + 2*FFFT + 2*np.log(M)*exp_U_V)*K + (3*K+1)*ADD + K*Comp + 4*K*U +2*COEF)*3 + 2*REF) + exp_U_V)*r  
    
    return cost

## Sublinear scaling and interaction picture babbush2019quantum

In [ ]:
def sublinear_scaling_interaction(N, eta, Gamma, lambd_T, lambd_U_V, eps_S, eps_HS, eps_PEA, eps_mu, eps_M_0):
    ''' 
    See the interaction_picture function for more background
    J represents the number of atoms
    In this article there are three additional sources of error, 
        - the precision on preparing the amplitudes sqrt(zeta_l), eps_mu
        - the precision on the position of the atomic nuclei, 1/delta_R. In the article we take log(1/delta_R) < 1/3 log(N)
        - The precision due to the finite value of M_0 = eta N t / eps_M_0
        
    The algorithm follows a very similar structure to that of the interaction_picture one. 
    '''
    
    t = 4.7/eps_PEA
    r = lambd_U_V*t # lambd_T is necessary to take tau = 1
    
    # Notice that K is a bit different than in other articles because each segment is now its own Taylor series, which has the consequence of larger error
    K = np.ceil( -1  + 2* np.log(2*r/epsilon_HS)/np.log(np.log(2*r/epsilon_HS))  ) # We 
    delta = eps_HS / t # Alternatively we can substitute t by r changing delta in the following line to 1/2. t represents L in the main text (see before eq 21 in the original article)
    tau = 1/np.ceil(2*lambd_U_V) # tau = t/ np.ceil(2 * lambd_T * t)
    M = np.max(16*tau/delta * (lambd_U_V + 2*lambd_T), K**2)
    M0 = eta * N * tau / (eps_M_0/r)
    
    rot_exp_T = np.log2(eta) + 2*np.log2(N)
    rot_select_1 = 1/3*np.log2(N) + 2
    rot_Subprepare  = 2 # Only the two rotations from Uniform in Subprepare
    rot_COEF = 2**(np.ceil(np.log2(K) + 1))
    
    num_rotations = (((2*np.log(M)*rot_exp_T + rot_select_1)*K + 2*rot_COEF)*3 + rot_exp_T )*r
    eps_SS = eps_S / num_rotations
    
    num_Subprepare = 2*3*K*3*r
    eps_mus = eps_mu / num_Subp
    
    Subprep = 4*J + 4*np.log(1/eps_mus) +8*np.log2(1/eps_SS)+  12*np.log2(J)
    n = 1/3*np.log2(N) + 1
    Prep  = 3*(79*n**2 +43*n*np.log2(M0) + 44*n)
    exp_T = rot_exp_T * 4*np.log(1/eps_SS)
    select_0 = 16*eta*np.log2(N)
    select_1 = 8*eta*np.log2(N) + 14*(np.log2(N))**2 + 4*np.log2(N)*np.log(1/eps_SS)
    
    HAM_T = 2*np.log(M)*exp_T + 2*(3*(Subprep + Prep)) + select_0 + select_1 #The 3 multiplying Subprep and Prep comes from oblivious AA
    U = 8*(np.log2(M) + np.log2(1/eps_SS))
    ADD = 4*np.log2(K)
    Comp = 8*np.log2(M)
    
    COEF = rot_COEF * (10 + 12*np.log2(K))
    REF = 16*(np.log2(Gamma) + 2*np.log(K+1)+ 2*np.log(M))
    
    cost = (((4*K*U + K*Comp + (3*K + 1)*ADD + K*HAM_T) + 2*COEF)*3  + 2*REF)*r
    
    # Initial state antisymmetrization
    antisymmetrization = 3*eta*np.log2(eta)*(np.log2(eta)-1)*(2* np.ceil(np.log2(eta**2)) + np.log(N))
    
    return cost + antisymmetrization

# Finding the molecule parameters

In [3]:
# Docs https://quantumai.google/reference/python/openfermion/

from openfermion.chem import geometry_from_pubchem, MolecularData
from openfermionpsi4 import run_psi4
from  openfermion.transforms  import  get_fermion_operator,  jordan_wigner
import openfermion
from openfermion.utils import Grid
from openfermion.hamiltonians import plane_wave_external_potential, plane_wave_potential, plane_wave_kinetic
from openfermion.hamiltonians import plane_wave_hamiltonian
from openfermion.hamiltonians import dual_basis_external_potential, dual_basis_potential, dual_basis_kinetic

from pyscf.mcscf import avas

methane_geometry = geometry_from_pubchem('methane')
print(methane_geometry)

basis = 'sto-3g'

molecule = MolecularData(methane_geometry, basis, multiplicity = 1)
print(molecule)

molecule = run_psi4(molecule,run_scf=True,
                        run_mp2=True,
                        #run_cisd=False,
                        #run_ccsd=True,
                        run_fci=False
                   )

[('C', (0, 0, 0)), ('H', (0.5541, 0.7996, 0.4965)), ('H', (0.6833, -0.8134, -0.2536)), ('H', (-0.7782, -0.3735, 0.6692)), ('H', (-0.4593, 0.3874, -0.9121))]


In [9]:
from pyscf import gto, scf, mcscf, fci,ao2mo
mol = gto.Mole()
mol = gto.M(
        atom = methane_geometry,
        basis = basis)
myhf = scf.RHF(mol) #.x2c() The x2c is relativistic. We are not so desperate :P
myhf.kernel()
mol.atom

converged SCF energy = -39.7265817118519


[('C', (0, 0, 0)),
 ('H', (0.5541, 0.7996, 0.4965)),
 ('H', (0.6833, -0.8134, -0.2536)),
 ('H', (-0.7782, -0.3735, 0.6692)),
 ('H', (-0.4593, 0.3874, -0.9121))]

In [17]:
ao_labels = ['C 2pz']
norb, ne_act, orbs = avas.avas(myhf, ao_labels, canonicalize=False)
mo_ints_myhf = ao2mo.kernel(mol, myhf.mo_coeff) #orbs)
print(mo_ints_myhf.shape)
mo_ints_orbs = ao2mo.kernel(mol, orbs)
print(mo_ints_orbs.shape)
print(mo_ints_myhf - mo_ints_orbs)

(45, 45)
(45, 45)
[[ 2.92982983e+00 -2.85459192e-01  5.91465974e-02 ... -1.11860112e-02
   2.66471566e-02  2.05217574e-01]
 [-2.85459192e-01  2.35658170e-02 -1.43562384e-01 ...  3.49209290e-07
   2.42649131e-04 -1.62842063e-02]
 [ 5.91465974e-02 -1.43562384e-01 -2.99766787e+00 ...  1.84095139e-06
  -7.74859929e-06 -2.61148750e-01]
 ...
 [-1.11860112e-02  3.49209289e-07  1.84095139e-06 ... -1.31244080e-05
   5.53431412e-03  1.39180374e-02]
 [ 2.66471566e-02  2.42649131e-04 -7.74859929e-06 ...  5.53431412e-03
   5.74825787e-02  6.73904843e-03]
 [ 2.05217574e-01 -1.62842063e-02 -2.61148750e-01 ...  1.39180374e-02
   6.73904843e-03 -7.40561615e-02]]


In [ ]:
'''
To obtain these Hamiltonians one must choose to study the system without a spin degree of freedom (spinless), 
one must the specify dimension in which the calculation is performed (n_dimensions, usually 3), 
one must specify how many plane waves are in each dimension (grid_length) 
and one must specify the length scale of the plane wave harmonics in each dimension (length_scale) 
and also the locations and charges of the nuclei.

Taken from https://quantumai.google/openfermion/tutorials/intro_to_openfermion
'''

grid = Grid(dimensions = 3, length = 8, scale = 1.) # La complejidad crece bastante con length
grid.volume_scale()

plane_wave_H = plane_wave_hamiltonian(grid, methane_geometry, True)
plane_wave_H 

In [ ]:
## Selection of active orbitals

ao_labels = ['Fe 3d', 'C 2pz']
norb, ne_act, orbs = avas.avas(mf, ao_labels, canonicalize=False)

In [ ]:
## Low rank approximation. See
'''
https://quantumai.google/openfermion/tutorials/circuits_3_arbitrary_basis_trotter

Low rank approximation: https://github.com/quantumlib/OpenFermion/blob/4781602e094699f0fe0844bcded8ef0d45653e81/src/openfermion/circuits/low_rank.py#L76

Ground state: https://quantumai.google/reference/python/openfermion/linalg/get_ground_state

On the integration of low rank calculations: https://github.com/quantumlib/OpenFermion/issues/708#issuecomment-777640581
'''

In [2]:
molecule = run_psi4(molecule,run_scf=True,
                        run_mp2=True,
                        run_cisd=False,
                        run_ccsd=True,
                        run_fci=False
                   )

In [5]:
dir(molecule)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_canonical_orbitals',
 '_ccsd_double_amps',
 '_ccsd_single_amps',
 '_cisd_one_rdm',
 '_cisd_two_rdm',
 '_fci_one_rdm',
 '_fci_two_rdm',
 '_one_body_integrals',
 '_overlap_integrals',
 '_two_body_integrals',
 'atoms',
 'basis',
 'canonical_orbitals',
 'ccsd_double_amps',
 'ccsd_energy',
 'ccsd_single_amps',
 'charge',
 'cisd_energy',
 'cisd_one_rdm',
 'cisd_two_rdm',
 'description',
 'fci_energy',
 'fci_one_rdm',
 'fci_two_rdm',
 'filename',
 'general_calculations',
 'geometry',
 'get_active_space_integrals',
 'get_antisym',
 'get_from_file',
 'get_integrals',
 'get_j',
 'get_k',
 'get_molecular_hamiltonian',
 'get_molecu

In [7]:
molecule.two_body_integrals

array([[[[ 3.50111808e+00,  2.93359989e-01,  1.10767442e-06, ...,
          -3.28313798e-05, -6.91012233e-05,  3.26772351e-01],
         [ 2.93359989e-01,  3.86008907e-02,  2.06425379e-07, ...,
          -4.34506685e-06, -9.16302869e-06,  4.38093713e-02],
         [ 1.10767442e-06,  2.06425379e-07,  8.77763273e-03, ...,
          -8.81953045e-05,  1.34410199e-02,  2.86638503e-06],
         ...,
         [-3.28313798e-05, -4.34506685e-06, -8.81953045e-05, ...,
           2.07111511e-02, -2.63509490e-08, -2.89584668e-06],
         [-6.91012233e-05, -9.16302869e-06,  1.34410199e-02, ...,
          -2.63509490e-08,  2.07122620e-02, -6.31082236e-06],
         [ 3.26772351e-01,  4.38093713e-02,  2.86638503e-06, ...,
          -2.89584668e-06, -6.31082236e-06,  4.97918460e-02]],

        [[ 2.93359989e-01,  3.86008907e-02,  2.06425379e-07, ...,
          -4.34506685e-06, -9.16302869e-06,  4.38093713e-02],
         [ 7.08049932e-01,  9.18605309e-03, -1.41229460e-07, ...,
          -1.13582449e

In [6]:
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
plane_waves_hamiltonian = openfermion.get_diagonal_coulomb_hamiltonian(fermionic_hamiltonian)
plane_waves_hamiltonian

ValueError: FermionOperator does not map to DiagonalCoulombHamiltonian (contains terms with indices (1, 0, 2, 1)).

In [1]:
from functools import reduce
import numpy
from pyscf import gto
from pyscf import scf
from pyscf import mcscf
from pyscf import fci


In [2]:
mol = gto.Mole()
mol.atom = '''
Fe     0.000000    0.000000    0.000000 
C     -0.713500   -0.982049   -1.648000 
C      0.713500   -0.982049   -1.648000 
C      1.154467    0.375109   -1.648000 
C      0.000000    1.213879   -1.648000 
C     -1.154467    0.375109   -1.648000 
H     -1.347694   -1.854942   -1.638208 
H      1.347694   -1.854942   -1.638208 
H      2.180615    0.708525   -1.638208 
H      0.000000    2.292835   -1.638208 
H     -2.180615    0.708525   -1.638208 
C     -0.713500   -0.982049    1.648000 
C     -1.154467    0.375109    1.648000 
C     -0.000000    1.213879    1.648000 
C      1.154467    0.375109    1.648000 
C      0.713500   -0.982049    1.648000 
H     -1.347694   -1.854942    1.638208 
H     -2.180615    0.708525    1.638208 
H      0.000000    2.292835    1.638208 
H      2.180615    0.708525    1.638208 
H      1.347694   -1.854942    1.638208  
'''
mol.basis = 'cc-pvtz-dk'
mol.spin = 0
mol.build()

In [4]:
mol.__dict__

{'output': None,
 'max_memory': 4000,
 'charge': 0,
 'spin': 0,
 'symmetry': False,
 'symmetry_subgroup': None,
 'cart': False,
 'atom': '\nFe     0.000000    0.000000    0.000000 \nC     -0.713500   -0.982049   -1.648000 \nC      0.713500   -0.982049   -1.648000 \nC      1.154467    0.375109   -1.648000 \nC      0.000000    1.213879   -1.648000 \nC     -1.154467    0.375109   -1.648000 \nH     -1.347694   -1.854942   -1.638208 \nH      1.347694   -1.854942   -1.638208 \nH      2.180615    0.708525   -1.638208 \nH      0.000000    2.292835   -1.638208 \nH     -2.180615    0.708525   -1.638208 \nC     -0.713500   -0.982049    1.648000 \nC     -1.154467    0.375109    1.648000 \nC     -0.000000    1.213879    1.648000 \nC      1.154467    0.375109    1.648000 \nC      0.713500   -0.982049    1.648000 \nH     -1.347694   -1.854942    1.638208 \nH     -2.180615    0.708525    1.638208 \nH      0.000000    2.292835    1.638208 \nH      2.180615    0.708525    1.638208 \nH      1.347694   -1

In [3]:
mf = scf.ROHF(mol)


In [ ]:
mf.kernel()

In [ ]:
from pyscf.mcscf import avas
# See also 43-dmet_cas.py and function gto.mole.search_ao_label for the rules
# of "ao_labels" in the following
ao_labels = ['Fe 3d', 'C 2pz']
norb, ne_act, orbs = avas.avas(mf, ao_labels, canonicalize=False)